In [1]:
!pip install psycopg2-binary
!apt install curl

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [110]:
import os
import psycopg2 as ps
import pandas as pd
from IPython.display import HTML
from datetime import date
import warnings
import matplotlib.pyplot as plt
import time
import requests
import re

warnings.filterwarnings('ignore')

conn = ps.connect(
    host=os.environ["DB_HOST"],
    port=os.environ["DB_PORT"],
    dbname=os.environ["DB_NAME"],
    user=os.environ["DB_USER_NAME"],
    password=os.environ["DB_PW"])

def has_lat_long(url):
    return '@' in url

def extractGpsStrFromUrl(longUrl):
    if longUrl is None:
        return None

    pattern="@-?[0-9]*([.][0-9]*)?,-?[0-9]*([.][0-9]*)?,-?[0-9]*([.][0-9].*)?z"  #"/@-?[0-9]*([.][0-9]*)?,-?[0-9]*([.][0-9]*)?,-?[0-9]*([.][0-9].*)?z/"
    matcher = re.compile(pattern)

    results = matcher.search(longUrl)
    
    if (results is not None):
        return longUrl[results.start():results.end()]
    else:
        return None

def get_all_venues():
    sql_get_all_venues = "SELECT * from venues"
    
    return pd.read_sql_query(sql_get_all_venues, con=conn)

def get_lat_lon(short_url):
    stream = os.popen("$(pwd)/get-lat-long.sh {}".format(short_url))
    return stream.read().rstrip()

def get_lat_lon_from_full_url(full_url):        
    stream = os.popen("$(pwd)/get-lat-long-from-full.sh '{}'".format(full_url))
    return stream.read().rstrip()

def getRedirectUrl(url):
    if url is not None and ('http://' in url or 'https://' in url):
        print("vist: ", url)
        response = requests.get(url)
        if response.history:
            response.history.reverse()
            return response.history[0].url
    
    return None

def split_lat_lon(latLonStr):
    print("latLongStr: ", latLonStr)
    components = latLonStr.split(",")
    lat = components[0].replace("@", "").strip()
    long = components[1].strip()
    altitude = components[2].replace('z', '').strip()
    
    return (lat, long, altitude)

venues = get_all_venues();
conn.close()

mapped_venues = []
unmapped_venues = []

print('found {} venues'.format(len(venues.values)))

ndx = 0
for venue in venues.values:
    print('processing {}/{} venues'.format(ndx, len(venues.values)))
    url = venue[4]
    guid = venue[0]
    name = venue[1]
    
    lat_lon_str = None

    if url is not None:
        lat_lon_str = extractGpsStrFromUrl(url)

        if lat_lon_str is None:
            redirect_url = getRedirectUrl(url)
            lat_lon_str = extractGpsStrFromUrl(redirect_url)
        
    # if url is not None and 'goo.gl' in url:
    #     lat_lon_str = get_lat_lon(url)
    # elif url is not None:
    #     full_url = get_lat_lon_from_full_url(url)
    #     lat_lon_str = get_lat_lon(full_url)
    # else:
    #     lat_lon_str = None

    if lat_lon_str is None:
         unmapped_venues.append({
            'id': guid,
            'name': name,
            'url': url,
            'gps': None,
            'latitude': None,
            'longitude': None,
            'altitude': None,
        })
    elif lat_lon_str.strip() == '':
         unmapped_venues.append({
            'id': guid,
            'name': name,
            'url': url,
            'gps': None,
            'latitude': None,
            'longitude': None,
            'altitude': None,
        })
    else:
        (lat, long, altitude) = split_lat_lon(lat_lon_str)

        mapped_venues.append({
            'id': guid,
            'name': name,
            'url': url,
            'gps': lat_lon_str,
            'latitude': lat,
            'longitude': long,
            'altitude': altitude,
        })

    ndx = ndx + 1
        

mapped_venues

found 467 venues
processing 0/467 venues
vist:  https://goo.gl/maps/PXBsHGVauTB2
latLongStr:  @38.048242,-84.494705,15z
processing 1/467 venues
vist:  https://goo.gl/maps/D63rj364Eok
latLongStr:  @38.029272,-84.479974,15z
processing 2/467 venues
vist:  https://goo.gl/maps/MerUrvdgk9u
latLongStr:  @38.0569816,-84.5276467,15z
processing 3/467 venues
vist:  https://goo.gl/maps/cFTbFbb7TmS2
latLongStr:  @38.0425773,-84.5088568,17z
processing 4/467 venues
vist:  https://goo.gl/maps/sWXpcbvH56o
latLongStr:  @38.0566852,-84.4839591,17z
processing 5/467 venues
vist:  https://goo.gl/maps/8j9WvZUJGkJ2
latLongStr:  @38.0503015,-84.5002392,15z
processing 6/467 venues
vist:  https://goo.gl/maps/k93t7ofU9jB2
latLongStr:  @38.0580866,-84.4823413,19z
processing 7/467 venues
vist:  https://goo.gl/maps/fcnsV7Mguvm
latLongStr:  @38.045547,-84.496316,15z
processing 8/467 venues
vist:  https://goo.gl/maps/Pxxm1cJXqmt
latLongStr:  @38.135889,-84.904752,17z
processing 9/467 venues
vist:  https://goo.gl/maps/

[{'id': 'd0858cc0-0478-11e8-808e-07b6d0fe568b',
  'name': 'Institute 193',
  'url': 'https://goo.gl/maps/PXBsHGVauTB2',
  'gps': '@38.048242,-84.494705,15z',
  'latitude': '38.048242',
  'longitude': '-84.494705',
  'altitude': '15'},
 {'id': '7025fedd-083b-4605-9847-016cf649feb9',
  'name': 'Henry Clay Estate',
  'url': 'https://goo.gl/maps/D63rj364Eok',
  'gps': '@38.029272,-84.479974,15z',
  'latitude': '38.029272',
  'longitude': '-84.479974',
  'altitude': '15'},
 {'id': '8e136d93-36a0-4317-a6d8-91ac62a063a6',
  'name': 'Broomwagon',
  'url': 'https://goo.gl/maps/MerUrvdgk9u',
  'gps': '@38.0569816,-84.5276467,15z',
  'latitude': '38.0569816',
  'longitude': '-84.5276467',
  'altitude': '15'},
 {'id': 'e0f0ee00-0479-11e8-808e-07b6d0fe568b',
  'name': 'Bolivar Art Gallery - UK School of Art',
  'url': 'https://goo.gl/maps/cFTbFbb7TmS2',
  'gps': '@38.0425773,-84.5088568,17z',
  'latitude': '38.0425773',
  'longitude': '-84.5088568',
  'altitude': '17'},
 {'id': 'd9ecdc6d-1e03-468a-

In [111]:
len(mapped_venues)

382

In [112]:
len(unmapped_venues)

85

In [113]:
import csv

keys = mapped_venues[0].keys()

with open('mapped_venues.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(mapped_venues)

In [114]:
import csv

keys = unmapped_venues[0].keys()

with open('unmapped_venues.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(unmapped_venues)